In [1]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from expected_vaep_model.config import exp_vaep_chain_output_path
from expected_vaep_model.visualization.explore import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load VAEP Data

In [2]:
exp_vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_match_chains_v5.csv")
exp_vaep_data.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type0,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Time_Since_Last_Action,Distance_Since_Last_Action,Chain_Duration,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,xGoals,xBehinds,xMiss,xGoals_normalised,xBehinds_normalised,xMiss_normalised,xScore,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Spoil_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Spoil_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Spoil_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,type_Carry_outcome_effective_a2,type_Carry_outcome_ineffective_a2,type_Carry_outcome_clanger_a2,type_Spoil_outcome_effective_a2,type_Spoil_outcome_ineffective_a2,type_Spoil_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,exp_scores_label,exp_concedes_label,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
0,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,NaN,NaN,0,0,Centre Bounce,False,True,False,False,False,H

In [3]:
exp_vaep_data[['exp_scores', 'exp_concedes']].sum()

exp_scores      814886.778911
exp_concedes    183148.595864
dtype: float64

In [4]:
exp_vaep_data[['exp_offensive_value', 'exp_defensive_value', 'exp_vaep_value']].sum()

exp_offensive_value    213415.507172
exp_defensive_value      4263.578565
exp_vaep_value         217679.085769
dtype: float64

In [6]:
exp_vaep_data.sort_values(by = 'exp_vaep_value', ascending=False).head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type0,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Time_Since_Last_Action,Distance_Since_Last_Action,Chain_Duration,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,xGoals,xBehinds,xMiss,xGoals_normalised,xBehinds_normalised,xMiss_normalised,xScore,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Spoil_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Spoil_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Spoil_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,type_Carry_outcome_effective_a2,type_Carry_outcome_ineffective_a2,type_Carry_outcome_clanger_a2,type_Spoil_outcome_effective_a2,type_Spoil_outcome_ineffective_a2,type_Spoil_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,exp_scores_label,exp_concedes_label,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
235674,29,possGain,goal,269.0,1,823,862.0,Gold Coast,Gold Coast,Ben King,Ben_King,Kick,61.0,0.0,effective,True,NaN,138,156,Brisbane Lions,Gold Coast,right,202119_BrisbaneLions_GoldCoast,202119,2021.0,NaN,1.0,NaN,0,6,Free For: Off The Ball,False,False,False,True,F

In [7]:
exp_vaep_data['Season'] = exp_vaep_data['match_id'].apply(lambda x: int(x[:4]))
exp_vaep_2021 = exp_vaep_data[exp_vaep_data['Season'] == 2021]
exp_vaep_2022 = exp_vaep_data[exp_vaep_data['Season'] == 2022]
exp_vaep_2023 = exp_vaep_data[exp_vaep_data['Season'] == 2023]

In [8]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-vaep-model/data/scored/scored_player_stats_v5.csv")

In [9]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [10]:
exp_vaep_data['exp_vaep_value'].max(), exp_vaep_data['exp_vaep_value'].min()

(5.791939, -4.430853)

Match Level

In [12]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [16]:
match_id = "2023F4_Collingwood_BrisbaneLions"

In [17]:
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
2,Bobby Hill,Collingwood,20.594895,22.409081,-1.814187,36,1.0,1.0,38.0,1.000000,1.000000,0.344994,2.929524,3.100138,-0.733744
27,Joe Daniher,Brisbane Lions,17.356716,18.512635,-1.155919,37,2.0,2.0,34.0,0.851035,0.823051,0.394074,2.256994,2.329216,-0.424824
28,Jordan De Goey,Collingwood,14.776815,14.105223,0.671592,37,3.0,6.0,13.0,0.732353,0.622897,0.530330,1.721180,1.457199,0.432816
36,Nick Daicos,Collingwood,14.570050,14.711261,-0.141211,58,4.0,4.0,22.0,0.722841,0.650419,0.469729,1.678238,1.577105,0.051372
22,Jamie Elliott,Collingwood,13.905052,13.289613,0.615439,30,5.0,8.0,14.0,0.692249,0.585858,0.526144,1.540126,1.295829,0.406463


In [18]:
match_noshot_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
2,Bobby Hill,Collingwood,20.594895,22.409081,-1.814187,36,1.0,1.0,38.0,1.000000,1.000000,0.344994,2.929524,3.100138,-0.733744
27,Joe Daniher,Brisbane Lions,17.356716,18.512635,-1.155919,37,2.0,2.0,34.0,0.851035,0.823051,0.394074,2.256994,2.329216,-0.424824
28,Jordan De Goey,Collingwood,14.776815,14.105223,0.671592,37,3.0,6.0,13.0,0.732353,0.622897,0.530330,1.721180,1.457199,0.432816
36,Nick Daicos,Collingwood,14.570050,14.711261,-0.141211,58,4.0,4.0,22.0,0.722841,0.650419,0.469729,1.678238,1.577105,0.051372
22,Jamie Elliott,Collingwood,13.905052,13.289613,0.615439,30,5.0,8.0,14.0,0.692249,0.585858,0.526144,1.540126,1.295829,0.406463


Player Ratings

In [19]:
players_vaep = get_player_vaep(exp_vaep_data)
players_vaep.sort_values('exp_vaep_value', ascending = False).head(5)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
5,Aaron Naughton,Western Bulldogs,1498.518580,1574.874472,-76.355893,1703
853,Tom Hawkins,Geelong,1449.736307,1526.545850,-76.809546,1831
271,Harry McKay,Carlton,1253.499332,1315.687604,-62.188273,1411
138,Charlie Cameron,Brisbane Lions,1199.694139,1263.066901,-63.372761,1662
433,Joe Daniher,Brisbane Lions,1171.608075,1242.350499,-70.742424,1931


In [22]:
players_noshot_vaep = get_player_vaep(exp_vaep_data[exp_vaep_data['Shot_At_Goal'] != True])
players_noshot_vaep.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
149,Christian Petracca,Melbourne,882.506679,957.257536,-74.750854,4050
584,Marcus Bontempelli,Western Bulldogs,873.295268,902.443145,-29.147875,3777
791,Shai Bolton,Richmond,728.155859,763.530173,-35.374313,2460
514,Lachie Neale,Brisbane Lions,686.594294,726.172567,-39.578272,3794
5,Aaron Naughton,Western Bulldogs,677.100179,695.231347,-18.131167,1420
858,Tom Liberatore,Western Bulldogs,653.929115,684.216413,-30.287298,3449
319,Jack Macrae,Western Bulldogs,653.729704,716.542308,-62.812604,4361
151,Clayton Oliver,Melbourne,642.434625,718.310305,-75.875681,4017
433,Joe Daniher,Brisbane Lions,632.315779,659.193875,-26.878096,1646
853,Tom Hawkins,Geelong,620.235878,638.138248,-17.902371,1507


Normalise for minutes played

In [23]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
856,Tom J. Lynch,Richmond,899.841294,944.666113,-44.824819,944,3282,37,27.417468,28.783245,-1.365778
5,Aaron Naughton,Western Bulldogs,1498.518580,1574.874472,-76.355893,1703,5928,70,25.278654,26.566708,-1.288055
616,Max King,St Kilda,1133.397913,1190.026257,-56.628343,1099,4541,53,24.959214,26.206260,-1.247046
429,Jesse Hogan,Greater Western Sydney,1042.835545,1095.208286,-52.372736,1371,4240,50,24.595178,25.830384,-1.235206
271,Harry McKay,Carlton,1253.499332,1315.687604,-62.188273,1411,5191,59,24.147550,25.345552,-1.198002
817,Taylor Walker,Adelaide,1144.787662,1208.130193,-63.342530,1569,4758,56,24.060270,25.391555,-1.331285
853,Tom Hawkins,Geelong,1449.736307,1526.545850,-76.809546,1831,6505,69,22.286492,23.467269,-1.180777
142,Charlie Curnow,Carlton,1043.950643,1111.735552,-67.784908,1437,4775,52,21.862841,23.282420,-1.419579
635,Mitch Lewis,Hawthorn,813.843528,849.491343,-35.647814,1077,3907,44,20.830395,21.742804,-0.912409
834,Toby Greene,Greater Western Sydney,1068.086210,1132.705084,-64.618873,1946,5184,57,20.603515,21.850021,-1.246506


In [24]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,873.295268,902.443145,-29.147875,3777,6008,71,14.535540,15.020691,-0.485151
149,Christian Petracca,Melbourne,882.506679,957.257536,-74.750854,4050,6443,74,13.697139,14.857326,-1.160187
702,Patrick Dangerfield,Geelong,479.297321,544.760117,-65.462795,2233,3615,52,13.258570,15.069436,-1.810866
791,Shai Bolton,Richmond,728.155859,763.530173,-35.374313,2460,5601,66,13.000462,13.632033,-0.631571
856,Tom J. Lynch,Richmond,410.393124,421.452211,-11.059088,770,3282,37,12.504361,12.841323,-0.336962
858,Tom Liberatore,Western Bulldogs,653.929115,684.216413,-30.287298,3449,5360,68,12.200170,12.765232,-0.565062
925,Zak Butters,Port Adelaide,533.875985,532.133468,1.742518,2688,4437,57,12.032364,11.993091,0.039272
437,Joel Selwood,Geelong,387.398674,409.281900,-21.883225,2052,3233,44,11.982638,12.659508,-0.676871
198,Darcy Parish,Essendon,554.286282,618.794168,-64.507886,3533,4636,57,11.956132,13.347588,-1.391456
151,Clayton Oliver,Melbourne,642.434625,718.310305,-75.875681,4017,5461,63,11.764047,13.153457,-1.389410


In [25]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
856,Tom J. Lynch,Richmond,899.841294,944.666113,-44.824819,944,3282,37,27.417468,28.783245,-1.365778
5,Aaron Naughton,Western Bulldogs,1498.518580,1574.874472,-76.355893,1703,5928,70,25.278654,26.566708,-1.288055
616,Max King,St Kilda,1133.397913,1190.026257,-56.628343,1099,4541,53,24.959214,26.206260,-1.247046
429,Jesse Hogan,Greater Western Sydney,1042.835545,1095.208286,-52.372736,1371,4240,50,24.595178,25.830384,-1.235206
817,Taylor Walker,Adelaide,1144.787662,1208.130193,-63.342530,1569,4758,56,24.060270,25.391555,-1.331285
271,Harry McKay,Carlton,1253.499332,1315.687604,-62.188273,1411,5191,59,24.147550,25.345552,-1.198002
853,Tom Hawkins,Geelong,1449.736307,1526.545850,-76.809546,1831,6505,69,22.286492,23.467269,-1.180777
142,Charlie Curnow,Carlton,1043.950643,1111.735552,-67.784908,1437,4775,52,21.862841,23.282420,-1.419579
834,Toby Greene,Greater Western Sydney,1068.086210,1132.705084,-64.618873,1946,5184,57,20.603515,21.850021,-1.246506
635,Mitch Lewis,Hawthorn,813.843528,849.491343,-35.647814,1077,3907,44,20.830395,21.742804,-0.912409


In [26]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
702,Patrick Dangerfield,Geelong,479.297321,544.760117,-65.462795,2233,3615,52,13.258570,15.069436,-1.810866
584,Marcus Bontempelli,Western Bulldogs,873.295268,902.443145,-29.147875,3777,6008,71,14.535540,15.020691,-0.485151
149,Christian Petracca,Melbourne,882.506679,957.257536,-74.750854,4050,6443,74,13.697139,14.857326,-1.160187
791,Shai Bolton,Richmond,728.155859,763.530173,-35.374313,2460,5601,66,13.000462,13.632033,-0.631571
198,Darcy Parish,Essendon,554.286282,618.794168,-64.507886,3533,4636,57,11.956132,13.347588,-1.391456
151,Clayton Oliver,Melbourne,642.434625,718.310305,-75.875681,4017,5461,63,11.764047,13.153457,-1.389410
701,Patrick Cripps,Carlton,595.715462,685.916846,-90.201385,3305,5321,65,11.195555,12.890751,-1.695196
856,Tom J. Lynch,Richmond,410.393124,421.452211,-11.059088,770,3282,37,12.504361,12.841323,-0.336962
873,Touk Miller,Gold Coast,525.534067,609.206009,-83.671941,3248,4753,56,11.056892,12.817295,-1.760403
858,Tom Liberatore,Western Bulldogs,653.929115,684.216413,-30.287298,3449,5360,68,12.200170,12.765232,-0.565062


In [27]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,472.385710,113.922497,358.463214,2152,5596,59,8.441489,2.035784,6.405704
264,Harris Andrews,Brisbane Lions,619.097459,211.963821,407.133638,2714,7103,73,8.716000,2.984145,5.731855
841,Tom Barrass,West Coast,370.042805,113.967076,256.075730,1791,4501,49,8.221346,2.532039,5.689307
845,Tom Clurey,Port Adelaide,159.675978,49.884568,109.791410,686,2210,25,7.225157,2.257220,4.967937
757,Sam Collins,Gold Coast,441.059024,136.007122,305.051901,2022,6175,65,7.142656,2.202544,4.940112
24,Aliir Aliir,Port Adelaide,512.143725,204.603316,307.540408,2208,6463,67,7.924241,3.165764,4.758478
861,Tom McCartin,Sydney,379.295410,117.287611,262.007801,1948,5712,62,6.640326,2.053355,4.586971
427,Jeremy McGovern,West Coast,221.858192,106.201374,115.656818,1249,2619,34,8.471103,4.055035,4.416068
304,Jack Buckley,Greater Western Sydney,273.874241,114.008169,159.866073,1303,3651,41,7.501349,3.122656,4.378693
197,Darcy Moore,Collingwood,495.512028,246.318393,249.193636,2141,5722,61,8.659770,4.304760,4.355009


In [28]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,472.385710,113.922497,358.463214,2152,5596,59,8.441489,2.035784,6.405704
264,Harris Andrews,Brisbane Lions,613.622373,205.813303,407.809070,2710,7103,73,8.638918,2.897555,5.741364
841,Tom Barrass,West Coast,368.224145,111.696954,256.527192,1790,4501,49,8.180941,2.481603,5.699338
845,Tom Clurey,Port Adelaide,159.675978,49.884568,109.791410,686,2210,25,7.225157,2.257220,4.967937
757,Sam Collins,Gold Coast,441.059024,136.007122,305.051901,2022,6175,65,7.142656,2.202544,4.940112
24,Aliir Aliir,Port Adelaide,499.173950,191.464602,307.709347,2204,6463,67,7.723564,2.962473,4.761092
861,Tom McCartin,Sydney,377.838423,115.633450,262.204976,1947,5712,62,6.614818,2.024395,4.590423
427,Jeremy McGovern,West Coast,221.615002,105.801444,115.813558,1247,2619,34,8.461818,4.039765,4.422053
197,Darcy Moore,Collingwood,465.427739,213.977463,251.450278,2132,5722,61,8.134005,3.739557,4.394447
304,Jack Buckley,Greater Western Sydney,269.661985,109.586123,160.075863,1300,3651,41,7.385976,3.001537,4.384439


VAEP by Action

In [29]:
action_vaep = get_vaep_action_summary(exp_vaep_data)
action_vaep.sort_values("exp_vaep_value_per_action", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Kick,79957.917480,96974.531908,-17016.614454,273657,0.285745,0.292183,0.354365,-0.062182,0.367320,0.454393,-3.991158
0,Carry,128872.690409,115329.850913,13542.839494,449629,0.469490,0.286620,0.256500,0.030120,0.592031,0.540401,3.176402
3,Spoil,11055.366404,793.159297,10262.207099,41852,0.043701,0.264154,0.018952,0.245202,0.050787,0.003717,2.406947
1,Handball,-2206.888524,317.965054,-2524.853574,192558,0.201064,-0.011461,0.001651,-0.013112,-0.010138,0.001490,-0.592191


In [30]:
match_data = get_match(exp_vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("exp_vaep_value_per_action", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Kick,126.739037,151.594319,-24.855282,427,0.312592,0.296813,0.355022,-0.058209,0.424565,0.488940,2.155477
0,Carry,164.230319,154.216566,10.013753,631,0.461933,0.260270,0.244400,0.015870,0.550157,0.497398,-0.868403
3,Spoil,12.748876,1.772330,10.976545,64,0.046852,0.199201,0.027693,0.171509,0.042708,0.005716,-0.951898
1,Handball,-5.202861,2.463377,-7.666238,244,0.178624,-0.021323,0.010096,-0.031419,-0.017429,0.007945,0.664824


In [31]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,788.330834,813.654511,-25.323677,2150,0.502689,0.366666,0.378444,-0.011778,0.710575,0.676128,0.269467
2,Kick,348.895329,403.889628,-54.994297,1117,0.261164,0.312350,0.361584,-0.049234,0.314483,0.335623,0.585189
3,Spoil,-1.983341,-2.567414,0.584073,18,0.004209,-0.110186,-0.142634,0.032449,-0.001788,-0.002133,-0.006215
1,Handball,-25.816139,-11.573045,-14.243093,992,0.231938,-0.026024,-0.011666,-0.014358,-0.023270,-0.009617,0.151559


In [32]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,260.526934,237.066415,23.460519,1217,0.475948,0.214073,0.194796,0.019277,0.774981,0.612487,-0.461064
2,Kick,95.553403,150.130505,-54.577103,718,0.280798,0.133083,0.209095,-0.076013,0.284240,0.387879,1.072591
3,Spoil,2.760933,2.445504,0.315429,8,0.003129,0.345117,0.305688,0.039429,0.008213,0.006318,-0.006199
1,Handball,-22.669389,-2.587143,-20.082246,614,0.240125,-0.036921,-0.004214,-0.032707,-0.067434,-0.006684,0.394672


In [33]:
player = "Darcy Moore"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,281.65,120.46,161.19,743,0.35,0.38,0.16,0.22,0.57,0.49,0.65
3,Spoil,186.07,5.74,180.33,407,0.19,0.46,0.01,0.44,0.38,0.02,0.72
1,Handball,16.18,7.35,8.83,342,0.16,0.05,0.02,0.03,0.03,0.03,0.04
2,Kick,11.61,112.76,-101.16,649,0.30,0.02,0.17,-0.16,0.02,0.46,-0.41


In [34]:
player = "Taylor Walker"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Kick,642.13,701.60,-59.47,538,0.34,1.19,1.30,-0.11,0.56,0.58,0.94
0,Carry,514.50,520.33,-5.84,763,0.49,0.67,0.68,-0.01,0.45,0.43,0.09
3,Spoil,0.13,-0.31,0.44,36,0.02,0.00,-0.01,0.01,0.00,-0.00,-0.01
1,Handball,-11.98,-13.49,1.52,232,0.15,-0.05,-0.06,0.01,-0.01,-0.01,-0.02


By Season

In [35]:
players_2021 = get_player_vaep(exp_vaep_2021)
players_2022 = get_player_vaep(exp_vaep_2022)
players_2023 = get_player_vaep(exp_vaep_2023)

In [36]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [37]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [38]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [39]:
player_ratings_2021.sort_values('exp_vaep_value', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
2,Aaron Naughton,Western Bulldogs,561.14,586.81,-25.68,595,2041,25,27.49,28.75,-1.26
613,Tom Hawkins,Geelong,478.41,507.56,-29.15,703,2304,24,20.76,22.03,-1.27
104,Charlie Dixon,Port Adelaide,464.92,489.07,-24.15,656,2276,24,20.43,21.49,-1.06
200,Harry McKay,Carlton,461.33,492.41,-31.08,403,1637,19,28.18,30.08,-1.90
312,Joe Daniher,Brisbane Lions,402.45,423.47,-21.02,701,2252,24,17.87,18.80,-0.93
240,Jack Riewoldt,Richmond,401.39,425.66,-24.27,497,2053,22,19.55,20.73,-1.18
100,Charlie Cameron,Brisbane Lions,399.34,420.05,-20.71,532,2082,24,19.18,20.18,-0.99
444,Max King,St Kilda,398.97,417.81,-18.84,406,1688,20,23.64,24.75,-1.12
420,Marcus Bontempelli,Western Bulldogs,386.75,395.46,-8.71,1487,2193,26,17.64,18.03,-0.40
224,Jack Darling,West Coast,382.00,396.90,-14.90,569,2129,22,17.94,18.64,-0.70


In [40]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
200,Harry McKay,Carlton,461.33,492.41,-31.08,403,1637,19,28.18,30.08,-1.90
2,Aaron Naughton,Western Bulldogs,561.14,586.81,-25.68,595,2041,25,27.49,28.75,-1.26
589,Taylor Walker,Adelaide,347.05,363.19,-16.14,436,1369,16,25.35,26.53,-1.18
444,Max King,St Kilda,398.97,417.81,-18.84,406,1688,20,23.64,24.75,-1.12
40,Ben Brown,Melbourne,239.36,248.65,-9.29,283,1037,13,23.08,23.98,-0.90
598,Toby Greene,Greater Western Sydney,354.10,378.52,-24.42,640,1627,18,21.76,23.26,-1.50
616,Tom J. Lynch,Richmond,357.65,374.11,-16.47,443,1681,18,21.28,22.26,-0.98
381,Lance Franklin,Sydney,339.66,359.20,-19.54,455,1615,18,21.03,22.24,-1.21
268,Jake Stringer,Essendon,320.61,338.19,-17.58,636,1526,19,21.01,22.16,-1.15
613,Tom Hawkins,Geelong,478.41,507.56,-29.15,703,2304,24,20.76,22.03,-1.27


In [41]:
player_ratings_2021.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
2,Aaron Naughton,Western Bulldogs,561.136032,586.813639,-25.677605,595,2041,25,27.493191,28.751281,-1.258089
613,Tom Hawkins,Geelong,478.411885,507.562963,-29.151077,703,2304,24,20.764405,22.029642,-1.265238
200,Harry McKay,Carlton,461.329951,492.413439,-31.083492,403,1637,19,28.181426,30.080235,-1.898808
104,Charlie Dixon,Port Adelaide,464.920717,489.072328,-24.151610,656,2276,24,20.427097,21.488239,-1.061143
240,Jack Riewoldt,Richmond,401.386296,425.656403,-24.270107,497,2053,22,19.551208,20.733385,-1.182178
312,Joe Daniher,Brisbane Lions,402.447660,423.468958,-21.021299,701,2252,24,17.870678,18.804128,-0.933450
100,Charlie Cameron,Brisbane Lions,399.341735,420.050647,-20.708911,532,2082,24,19.180679,20.175343,-0.994664
444,Max King,St Kilda,398.972902,417.812081,-18.839177,406,1688,20,23.635835,24.751901,-1.116065
110,Christian Petracca,Melbourne,350.308637,403.091522,-52.782884,1485,2237,25,15.659751,18.019290,-2.359539
224,Jack Darling,West Coast,381.997131,396.898376,-14.901246,569,2129,22,17.942561,18.642479,-0.699918


In [42]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
200,Harry McKay,Carlton,461.329951,492.413439,-31.083492,403,1637,19,28.181426,30.080235,-1.898808
2,Aaron Naughton,Western Bulldogs,561.136032,586.813639,-25.677605,595,2041,25,27.493191,28.751281,-1.258089
589,Taylor Walker,Adelaide,347.049045,363.193157,-16.144113,436,1369,16,25.350551,26.529814,-1.179263
444,Max King,St Kilda,398.972902,417.812081,-18.839177,406,1688,20,23.635835,24.751901,-1.116065
40,Ben Brown,Melbourne,239.359907,248.648793,-9.288887,283,1037,13,23.081958,23.977704,-0.895746
598,Toby Greene,Greater Western Sydney,354.097749,378.519304,-24.421557,640,1627,18,21.763844,23.264862,-1.501018
616,Tom J. Lynch,Richmond,357.647731,374.114715,-16.466984,443,1681,18,21.275891,22.255486,-0.979595
381,Lance Franklin,Sydney,339.661439,359.196681,-19.535241,455,1615,18,21.031668,22.241281,-1.209612
268,Jake Stringer,Essendon,320.608791,338.186882,-17.578092,636,1526,19,21.009750,22.161657,-1.151906
613,Tom Hawkins,Geelong,478.411885,507.562963,-29.151077,703,2304,24,20.764405,22.029642,-1.265238


In [43]:
player_ratings_2021.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,247.148592,79.788078,167.360513,913,2224,23,11.112796,3.587593,7.525203
263,Jake Lever,Melbourne,209.763503,86.640087,123.123416,921,2472,25,8.485579,3.504858,4.980721
557,Sam Taylor,Greater Western Sydney,149.215762,35.387624,113.828138,691,1781,19,8.378201,1.986952,6.391249
18,Aliir Aliir,Port Adelaide,185.394097,89.440438,95.953657,858,2315,24,8.008384,3.863518,4.144866
540,Sam Collins,Gold Coast,133.383739,37.540580,95.843160,615,1788,20,7.459941,2.099585,5.360356
617,Tom Jonas,Port Adelaide,151.091779,59.138005,91.953773,815,2297,24,6.577788,2.574576,4.003212
228,Jack Henry,Geelong,201.569700,112.468925,89.100775,788,2279,24,8.844656,4.935012,3.909643
13,Alex Keath,Western Bulldogs,146.289864,60.941378,85.348487,678,2173,23,6.732161,2.804481,3.927680
391,Liam Jones,Carlton,150.807320,65.563463,85.243856,579,1814,19,8.313524,3.614303,4.699220
253,Jacob Weitering,Carlton,161.010135,81.457682,79.552454,876,2110,22,7.630812,3.860554,3.770258


In [44]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,247.148592,79.788078,167.360513,913,2224,23,11.112796,3.587593,7.525203
557,Sam Taylor,Greater Western Sydney,149.215762,35.387624,113.828138,691,1781,19,8.378201,1.986952,6.391249
540,Sam Collins,Gold Coast,133.383739,37.540580,95.843160,615,1788,20,7.459941,2.099585,5.360356
604,Tom Barrass,West Coast,107.692314,36.859579,70.832735,563,1393,16,7.730963,2.646057,5.084906
263,Jake Lever,Melbourne,209.763503,86.640087,123.123416,921,2472,25,8.485579,3.504858,4.980721
308,Jeremy McGovern,West Coast,112.699768,53.169755,59.530013,556,1204,15,9.360446,4.416093,4.944353
73,Brennan Cox,Fremantle,77.967426,25.958785,52.008642,395,1085,12,7.185938,2.392515,4.793423
391,Liam Jones,Carlton,150.807320,65.563463,85.243856,579,1814,19,8.313524,3.614303,4.699220
610,Tom Doedee,Adelaide,143.151162,63.799831,79.351331,774,1785,20,8.019673,3.574220,4.445453
144,Darcy Moore,Collingwood,128.684199,77.372573,51.311626,488,1211,13,10.626276,6.389147,4.237128


2022

In [45]:
player_ratings_2022.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
623,Tom Hawkins,Geelong,563.261789,591.140494,-27.878706,687,2366,25,23.806500,24.984805,-1.178305
626,Tom J. Lynch,Richmond,542.193563,570.551398,-28.357835,501,1601,19,33.865931,35.637189,-1.771258
456,Max King,St Kilda,504.076643,532.502064,-28.425419,486,2008,22,25.103418,26.519027,-1.415609
311,Jeremy Cameron,Geelong,473.325813,499.214037,-25.888224,810,2214,24,21.378763,22.548060,-1.169296
3,Aaron Naughton,Western Bulldogs,462.208881,488.504302,-26.295422,533,1899,22,24.339594,25.724292,-1.384698
211,Isaac Heeney,Sydney,433.404432,456.039768,-22.635335,859,2136,25,20.290470,21.350176,-1.059707
104,Charlie Curnow,Carlton,421.117135,447.084346,-25.967210,573,2022,22,20.826762,22.110996,-1.284234
521,Peter Wright,Essendon,420.452431,448.875933,-28.423501,572,2162,22,19.447383,20.762069,-1.314686
194,Harry McKay,Carlton,415.633583,434.772732,-19.139147,469,1746,19,23.804902,24.901073,-1.096171
582,Shai Bolton,Richmond,394.571938,415.332863,-20.760922,840,1985,23,19.877680,20.923570,-1.045890


In [46]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,542.193563,570.551398,-28.357835,501,1601,19,33.865931,35.637189,-1.771258
456,Max King,St Kilda,504.076643,532.502064,-28.425419,486,2008,22,25.103418,26.519027,-1.415609
3,Aaron Naughton,Western Bulldogs,462.208881,488.504302,-26.295422,533,1899,22,24.339594,25.724292,-1.384698
623,Tom Hawkins,Geelong,563.261789,591.140494,-27.878706,687,2366,25,23.806500,24.984805,-1.178305
194,Harry McKay,Carlton,415.633583,434.772732,-19.139147,469,1746,19,23.804902,24.901073,-1.096171
316,Jesse Hogan,Greater Western Sydney,355.580303,374.272713,-18.692408,490,1505,18,23.626598,24.868619,-1.242020
599,Taylor Walker,Adelaide,360.264153,379.227321,-18.963167,540,1566,18,23.005374,24.216304,-1.210930
311,Jeremy Cameron,Geelong,473.325813,499.214037,-25.888224,810,2214,24,21.378763,22.548060,-1.169296
427,Mabior Chol,Gold Coast,374.516863,391.444693,-16.927830,417,1777,22,21.075794,22.028401,-0.952607
104,Charlie Curnow,Carlton,421.117135,447.084346,-25.967210,573,2022,22,20.826762,22.110996,-1.284234


In [47]:
player_ratings_2022.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
623,Tom Hawkins,Geelong,563.261789,591.140494,-27.878706,687,2366,25,23.806500,24.984805,-1.178305
626,Tom J. Lynch,Richmond,542.193563,570.551398,-28.357835,501,1601,19,33.865931,35.637189,-1.771258
456,Max King,St Kilda,504.076643,532.502064,-28.425419,486,2008,22,25.103418,26.519027,-1.415609
311,Jeremy Cameron,Geelong,473.325813,499.214037,-25.888224,810,2214,24,21.378763,22.548060,-1.169296
3,Aaron Naughton,Western Bulldogs,462.208881,488.504302,-26.295422,533,1899,22,24.339594,25.724292,-1.384698
211,Isaac Heeney,Sydney,433.404432,456.039768,-22.635335,859,2136,25,20.290470,21.350176,-1.059707
521,Peter Wright,Essendon,420.452431,448.875933,-28.423501,572,2162,22,19.447383,20.762069,-1.314686
104,Charlie Curnow,Carlton,421.117135,447.084346,-25.967210,573,2022,22,20.826762,22.110996,-1.284234
194,Harry McKay,Carlton,415.633583,434.772732,-19.139147,469,1746,19,23.804902,24.901073,-1.096171
582,Shai Bolton,Richmond,394.571938,415.332863,-20.760922,840,1985,23,19.877680,20.923570,-1.045890


In [48]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,542.193563,570.551398,-28.357835,501,1601,19,33.865931,35.637189,-1.771258
456,Max King,St Kilda,504.076643,532.502064,-28.425419,486,2008,22,25.103418,26.519027,-1.415609
3,Aaron Naughton,Western Bulldogs,462.208881,488.504302,-26.295422,533,1899,22,24.339594,25.724292,-1.384698
623,Tom Hawkins,Geelong,563.261789,591.140494,-27.878706,687,2366,25,23.806500,24.984805,-1.178305
194,Harry McKay,Carlton,415.633583,434.772732,-19.139147,469,1746,19,23.804902,24.901073,-1.096171
316,Jesse Hogan,Greater Western Sydney,355.580303,374.272713,-18.692408,490,1505,18,23.626598,24.868619,-1.242020
599,Taylor Walker,Adelaide,360.264153,379.227321,-18.963167,540,1566,18,23.005374,24.216304,-1.210930
311,Jeremy Cameron,Geelong,473.325813,499.214037,-25.888224,810,2214,24,21.378763,22.548060,-1.169296
104,Charlie Curnow,Carlton,421.117135,447.084346,-25.967210,573,2022,22,20.826762,22.110996,-1.284234
427,Mabior Chol,Gold Coast,374.516863,391.444693,-16.927830,417,1777,22,21.075794,22.028401,-0.952607


In [49]:
player_ratings_2022.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
573,Sam Taylor,Greater Western Sydney,162.315305,37.627217,124.688089,849,2132,22,7.613288,1.764879,5.848409
555,Sam Collins,Gold Coast,168.054518,48.112672,119.941845,666,2154,22,7.801974,2.233643,5.568331
144,Darcy Moore,Collingwood,192.103063,78.468691,113.634373,808,2314,24,8.301775,3.391041,4.910733
615,Tom Barrass,West Coast,150.139786,45.134131,105.005656,738,1767,19,8.496875,2.554280,5.942595
188,Harris Andrews,Brisbane Lions,151.810935,47.424145,104.386790,818,2337,24,6.495975,2.029274,4.466700
629,Tom McCartin,Sydney,143.771211,41.285924,102.485288,699,2351,25,6.115322,1.756101,4.359221
551,Ryan Gardner,Western Bulldogs,136.102396,34.797254,101.305142,679,2152,22,6.324461,1.616973,4.707488
92,Callum Wilkie,St Kilda,175.169317,78.111915,97.057402,846,2175,22,8.053762,3.591352,4.462409
558,Sam De Koning,Geelong,135.818625,40.630024,95.188601,687,2114,23,6.424722,1.921950,4.502772
255,Jacob Weitering,Carlton,133.572049,41.438652,92.133398,551,1680,18,7.950717,2.466586,5.484131


In [50]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
615,Tom Barrass,West Coast,150.139786,45.134131,105.005656,738,1767,19,8.496875,2.554280,5.942595
573,Sam Taylor,Greater Western Sydney,162.315305,37.627217,124.688089,849,2132,22,7.613288,1.764879,5.848409
555,Sam Collins,Gold Coast,168.054518,48.112672,119.941845,666,2154,22,7.801974,2.233643,5.568331
255,Jacob Weitering,Carlton,133.572049,41.438652,92.133398,551,1680,18,7.950717,2.466586,5.484131
617,Tom Clurey,Port Adelaide,84.522300,24.560056,59.962244,335,1182,13,7.150787,2.077839,5.072948
19,Aliir Aliir,Port Adelaide,138.765655,50.510622,88.255033,622,1797,19,7.722073,2.810830,4.911243
144,Darcy Moore,Collingwood,192.103063,78.468691,113.634373,808,2314,24,8.301775,3.391041,4.910733
353,Josh Walker,North Melbourne,98.203648,31.935989,66.267659,426,1371,16,7.162921,2.329394,4.833527
162,Dylan Grimes,Richmond,86.178764,27.674400,58.504364,386,1228,15,7.017815,2.253616,4.764199
551,Ryan Gardner,Western Bulldogs,136.102396,34.797254,101.305142,679,2152,22,6.324461,1.616973,4.707488


2023

In [51]:
player_ratings_2023.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
105,Charlie Curnow,Carlton,590.174461,629.019015,-38.844552,761,2437,26,24.217253,25.811203,-1.593950
320,Joe Daniher,Brisbane Lions,505.186120,537.917389,-32.731267,807,2297,26,21.993301,23.418258,-1.424957
316,Jesse Hogan,Greater Western Sydney,494.365610,516.905703,-22.540091,637,1984,23,24.917621,26.053715,-1.136093
3,Aaron Naughton,Western Bulldogs,475.173666,499.556531,-24.382866,575,1988,23,23.902096,25.128598,-1.226502
486,Nick Larkey,North Melbourne,453.815602,483.924989,-30.109389,486,2215,23,20.488289,21.847629,-1.359340
596,Toby Greene,Greater Western Sydney,451.446390,475.778121,-24.331729,871,2166,24,20.842400,21.965749,-1.123349
102,Charlie Cameron,Brisbane Lions,438.420040,457.151622,-18.731583,540,2136,26,20.525283,21.402229,-0.876947
585,Taylor Walker,Adelaide,437.474464,465.709714,-28.235250,593,1823,22,23.997502,25.546337,-1.548834
611,Tom Hawkins,Geelong,408.062633,427.842394,-19.779763,441,1835,20,22.237746,23.315662,-1.077916
111,Christian Petracca,Melbourne,404.312613,421.865587,-17.552972,1425,2098,25,19.271335,20.107988,-0.836653


In [52]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).round(2).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
316,Jesse Hogan,Greater Western Sydney,494.37,516.91,-22.54,637,1984,23,24.92,26.05,-1.14
105,Charlie Curnow,Carlton,590.17,629.02,-38.84,761,2437,26,24.22,25.81,-1.59
585,Taylor Walker,Adelaide,437.47,465.71,-28.24,593,1823,22,24.00,25.55,-1.55
3,Aaron Naughton,Western Bulldogs,475.17,499.56,-24.38,575,1988,23,23.90,25.13,-1.23
462,Mitch Lewis,Hawthorn,312.69,326.58,-13.88,405,1328,15,23.55,24.59,-1.05
313,Jeremy Finlayson,Port Adelaide,402.71,425.09,-22.38,604,1798,22,22.40,23.64,-1.24
611,Tom Hawkins,Geelong,408.06,427.84,-19.78,441,1835,20,22.24,23.32,-1.08
320,Joe Daniher,Brisbane Lions,505.19,537.92,-32.73,807,2297,26,21.99,23.42,-1.42
601,Todd Marshall,Port Adelaide,360.14,377.35,-17.21,420,1654,21,21.77,22.81,-1.04
48,Ben King,Gold Coast,367.09,385.94,-18.86,337,1694,20,21.67,22.78,-1.11


In [53]:
player_ratings_2023.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
105,Charlie Curnow,Carlton,590.174461,629.019015,-38.844552,761,2437,26,24.217253,25.811203,-1.593950
320,Joe Daniher,Brisbane Lions,505.186120,537.917389,-32.731267,807,2297,26,21.993301,23.418258,-1.424957
316,Jesse Hogan,Greater Western Sydney,494.365610,516.905703,-22.540091,637,1984,23,24.917621,26.053715,-1.136093
3,Aaron Naughton,Western Bulldogs,475.173666,499.556531,-24.382866,575,1988,23,23.902096,25.128598,-1.226502
486,Nick Larkey,North Melbourne,453.815602,483.924989,-30.109389,486,2215,23,20.488289,21.847629,-1.359340
596,Toby Greene,Greater Western Sydney,451.446390,475.778121,-24.331729,871,2166,24,20.842400,21.965749,-1.123349
585,Taylor Walker,Adelaide,437.474464,465.709714,-28.235250,593,1823,22,23.997502,25.546337,-1.548834
102,Charlie Cameron,Brisbane Lions,438.420040,457.151622,-18.731583,540,2136,26,20.525283,21.402229,-0.876947
611,Tom Hawkins,Geelong,408.062633,427.842394,-19.779763,441,1835,20,22.237746,23.315662,-1.077916
313,Jeremy Finlayson,Port Adelaide,402.708453,425.085389,-22.376937,604,1798,22,22.397578,23.642124,-1.244546


In [54]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
316,Jesse Hogan,Greater Western Sydney,494.365610,516.905703,-22.540091,637,1984,23,24.917621,26.053715,-1.136093
105,Charlie Curnow,Carlton,590.174461,629.019015,-38.844552,761,2437,26,24.217253,25.811203,-1.593950
585,Taylor Walker,Adelaide,437.474464,465.709714,-28.235250,593,1823,22,23.997502,25.546337,-1.548834
3,Aaron Naughton,Western Bulldogs,475.173666,499.556531,-24.382866,575,1988,23,23.902096,25.128598,-1.226502
462,Mitch Lewis,Hawthorn,312.693242,326.575615,-13.882372,405,1328,15,23.546178,24.591537,-1.045359
313,Jeremy Finlayson,Port Adelaide,402.708453,425.085389,-22.376937,604,1798,22,22.397578,23.642124,-1.244546
320,Joe Daniher,Brisbane Lions,505.186120,537.917389,-32.731267,807,2297,26,21.993301,23.418258,-1.424957
611,Tom Hawkins,Geelong,408.062633,427.842394,-19.779763,441,1835,20,22.237746,23.315662,-1.077916
601,Todd Marshall,Port Adelaide,360.138759,377.347195,-17.208435,420,1654,21,21.773806,22.814220,-1.040413
48,Ben King,Gold Coast,367.086049,385.941507,-18.855459,337,1694,20,21.669779,22.782852,-1.113073


In [55]:
player_ratings_2023.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
193,Harris Andrews,Brisbane Lions,220.137932,84.751598,135.386334,983,2542,26,8.660029,3.334052,5.325977
226,Jack Buckley,Greater Western Sydney,200.368992,75.374164,124.994828,894,2426,26,8.259233,3.106932,5.152301
18,Aliir Aliir,Port Adelaide,187.983973,64.652255,123.331718,728,2351,24,7.995915,2.749990,5.245926
562,Sam Taylor,Greater Western Sydney,160.854644,40.907656,119.946987,612,1683,18,9.557614,2.430639,7.126975
116,Connor Idun,Greater Western Sydney,185.497432,75.171914,110.325518,1022,2298,26,8.072125,3.271189,4.800936
91,Callum Wilkie,St Kilda,173.151920,67.366345,105.785574,1063,2385,24,7.260039,2.824585,4.435454
101,Charlie Ballard,Gold Coast,158.864191,54.547753,104.316438,707,2202,23,7.214541,2.477191,4.737350
261,Jacob Weitering,Carlton,174.636529,72.257015,102.379514,959,2567,26,6.803137,2.814843,3.988294
69,Brandon Zerk-Thatcher,Essendon,134.022092,34.142754,99.879337,677,2004,22,6.687729,1.703730,4.983999
616,Tom McCartin,Sydney,130.075791,30.737562,99.338228,509,1310,15,9.929450,2.346379,7.583071


In [56]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
616,Tom McCartin,Sydney,130.075791,30.737562,99.338228,509,1310,15,9.929450,2.346379,7.583071
562,Sam Taylor,Greater Western Sydney,160.854644,40.907656,119.946987,612,1683,18,9.557614,2.430639,7.126975
603,Tom Barrass,West Coast,112.210705,31.973366,80.237339,490,1341,14,8.367689,2.384293,5.983396
487,Nick Murray,Adelaide,119.930655,42.851000,77.079656,445,1353,17,8.864054,3.167110,5.696944
193,Harris Andrews,Brisbane Lions,220.137932,84.751598,135.386334,983,2542,26,8.660029,3.334052,5.325977
18,Aliir Aliir,Port Adelaide,187.983973,64.652255,123.331718,728,2351,24,7.995915,2.749990,5.245926
226,Jack Buckley,Greater Western Sydney,200.368992,75.374164,124.994828,894,2426,26,8.259233,3.106932,5.152301
69,Brandon Zerk-Thatcher,Essendon,134.022092,34.142754,99.879337,677,2004,22,6.687729,1.703730,4.983999
116,Connor Idun,Greater Western Sydney,185.497432,75.171914,110.325518,1022,2298,26,8.072125,3.271189,4.800936
101,Charlie Ballard,Gold Coast,158.864191,54.547753,104.316438,707,2202,23,7.214541,2.477191,4.737350


In [58]:
action_vaep = get_vaep_action_summary(exp_vaep_2021)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
1,Handball,-665.312630,35.034669,-700.347297,63578,0.201002,-0.010465,0.000551,-0.011016,-0.009484,0.000511,-0.454095
3,Spoil,3393.648955,231.332325,3162.316629,13587,0.042955,0.249772,0.017026,0.232746,0.048376,0.003372,2.050402
0,Carry,41698.592324,37076.125528,4622.466785,148666,0.470007,0.280485,0.249392,0.031093,0.594403,0.540392,2.997143
2,Kick,25725.075215,31267.220347,-5542.145157,90475,0.286036,0.284334,0.345590,-0.061256,0.366705,0.455726,-3.593450


In [59]:
action_vaep = get_vaep_action_summary(exp_vaep_2022)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
1,Handball,-639.063590,179.116254,-818.179843,62101,0.198426,-0.010291,0.002884,-0.013175,-0.008914,0.002547,-0.599437
3,Spoil,3716.129813,247.836669,3468.293142,13959,0.044602,0.266217,0.017755,0.248463,0.051834,0.003524,2.541036
0,Carry,42332.260757,37987.330571,4344.930198,146874,0.469294,0.288222,0.258639,0.029583,0.590467,0.540145,3.183302
2,Kick,26283.566272,31913.696894,-5630.130633,90034,0.287678,0.291929,0.354463,-0.062533,0.366613,0.453784,-4.124901


In [60]:
action_vaep = get_vaep_action_summary(exp_vaep_2023)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
1,Handball,-902.512304,103.814131,-1006.326434,66879,0.203637,-0.013495,0.001552,-0.015047,-0.011901,0.001394,-0.741924
3,Spoil,3945.587636,313.990303,3631.597328,14306,0.043560,0.275799,0.021948,0.253851,0.052029,0.004216,2.677429
0,Carry,44841.837328,40266.394815,4575.442511,154089,0.469180,0.291013,0.261319,0.029694,0.591314,0.540650,3.373288
2,Kick,27949.275993,33793.614667,-5844.338665,93148,0.283623,0.300052,0.362795,-0.062743,0.368558,0.453741,-4.308794


In [61]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,103.46,85.43,18.03,599,0.48,0.17,0.14,0.03,0.83,0.58,-0.80
2,Kick,40.31,67.27,-26.96,361,0.29,0.11,0.19,-0.07,0.32,0.46,1.20
3,Spoil,1.38,1.13,0.25,5,0.00,0.28,0.23,0.05,0.01,0.01,-0.01
1,Handball,-20.08,-6.31,-13.78,286,0.23,-0.07,-0.02,-0.05,-0.16,-0.04,0.61


In [62]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,157.06,151.64,5.43,618,0.47,0.25,0.25,0.01,0.74,0.63,-0.19
2,Kick,55.24,82.86,-27.61,357,0.27,0.15,0.23,-0.08,0.26,0.35,0.97
3,Spoil,1.38,1.31,0.07,3,0.00,0.46,0.44,0.02,0.01,0.01,-0.00
1,Handball,-2.58,3.72,-6.30,328,0.25,-0.01,0.01,-0.02,-0.01,0.02,0.22


In [63]:
player_ratings_2023[player_ratings_2023['player'] == 'Nick Daicos']

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
482,Nick Daicos,Collingwood,211.104633,239.526263,-28.42163,1306,1854,22,11.386442,12.919432,-1.53299


In [64]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,260.64,263.89,-3.25,685,0.50,0.38,0.39,-0.00,0.73,0.70,0.14
2,Kick,103.29,118.76,-15.47,357,0.26,0.29,0.33,-0.04,0.29,0.31,0.65
3,Spoil,-2.06,-1.89,-0.17,5,0.00,-0.41,-0.38,-0.03,-0.01,-0.00,0.01
1,Handball,-7.06,-2.31,-4.75,320,0.23,-0.02,-0.01,-0.01,-0.02,-0.01,0.20


In [65]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,272.08,271.92,0.17,722,0.51,0.38,0.38,0.00,0.67,0.64,-0.01
2,Kick,133.34,147.40,-14.05,351,0.25,0.38,0.42,-0.04,0.33,0.35,0.80
3,Spoil,-0.23,-0.84,0.60,6,0.00,-0.04,-0.14,0.10,-0.00,-0.00,-0.03
1,Handball,-0.88,3.39,-4.27,346,0.24,-0.00,0.01,-0.01,-0.00,0.01,0.24


In [66]:
player_ratings_2023[player_ratings_2023['player'] ==  "Christian Petracca"]

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
111,Christian Petracca,Melbourne,404.312613,421.865587,-17.552972,1425,2098,25,19.271335,20.107988,-0.836653


In [67]:
team_data = get_team(exp_vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,2747.36,2471.54,275.82,9271,0.47,0.30,0.27,0.03,0.58,0.54,2.13
2,Kick,1829.71,2128.04,-298.33,5541,0.28,0.33,0.38,-0.05,0.39,0.46,-2.31
3,Spoil,206.82,15.32,191.50,762,0.04,0.27,0.02,0.25,0.04,0.00,1.48
1,Handball,-49.04,-9.46,-39.57,4045,0.21,-0.01,-0.00,-0.01,-0.01,-0.00,-0.31


In [68]:
team_data = get_team(exp_vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,2396.25,2167.17,229.08,7744,0.47,0.31,0.28,0.03,0.58,0.53,9.09
2,Kick,1579.12,1898.98,-319.86,4873,0.30,0.32,0.39,-0.07,0.38,0.46,-12.69
3,Spoil,210.63,26.79,183.84,668,0.04,0.32,0.04,0.28,0.05,0.01,7.30
1,Handball,-67.44,0.41,-67.86,3215,0.19,-0.02,0.00,-0.02,-0.02,0.00,-2.69


In [69]:
team_data = get_team(exp_vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,2708.55,2488.10,220.44,9217,0.47,0.29,0.27,0.02,0.57,0.52,-18.70
2,Kick,1817.78,2217.90,-400.12,5568,0.28,0.33,0.40,-0.07,0.38,0.47,33.95
3,Spoil,236.56,16.25,220.31,823,0.04,0.29,0.02,0.27,0.05,0.00,-18.69
1,Handball,-9.08,43.34,-52.42,4163,0.21,-0.00,0.01,-0.01,-0.00,0.01,4.45


In [70]:
team_data = get_team(exp_vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value", ascending=False).round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
0,Carry,2541.06,2251.36,289.70,9222,0.47,0.28,0.24,0.03,0.62,0.56,2.37
2,Kick,1468.34,1780.07,-311.73,5580,0.29,0.26,0.32,-0.06,0.36,0.45,-2.55
3,Spoil,212.76,13.67,199.09,732,0.04,0.29,0.02,0.27,0.05,0.00,1.63
1,Handball,-100.10,-45.12,-54.98,4008,0.21,-0.02,-0.01,-0.01,-0.02,-0.01,-0.45


Match Expectd Scores v Expected VAEP

In [71]:
match_id = "2022F4_Geelong_Sydney"
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
37,Tom Hawkins,Geelong,27.296088,28.795222,-1.499135,30,1.0,1.0,41.0,1.000000,1.000000,0.276387,2.942110,3.069283,-0.985542
30,Patrick Dangerfield,Geelong,23.963091,24.708626,-0.745535,55,2.0,2.0,31.0,0.884468,0.859864,0.373684,2.454823,2.493548,-0.538724
5,Chad Warner,Sydney,21.060939,19.525263,1.535676,60,3.0,4.0,10.0,0.783870,0.682119,0.668213,2.030525,1.763298,0.813832
13,Isaac Smith,Geelong,20.644585,21.851999,-1.207413,65,4.0,3.0,38.0,0.769437,0.761906,0.314051,1.969654,2.091097,-0.812577
24,Mark Blicavs,Geelong,20.234983,19.097522,1.137461,48,5.0,5.0,12.0,0.755239,0.667451,0.616799,1.909770,1.703036,0.577726


In [72]:
match_chain = get_match(exp_vaep_data, match_id)
match_chain.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,ballUp,centreBounce,kickIn,possGain,throwIn,Event_Type0,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Time_Since_Last_Action,Distance_Since_Last_Action,Chain_Duration,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,xGoals,xBehinds,xMiss,xGoals_normalised,xBehinds_normalised,xMiss_normalised,xScore,action_type,Contested,Mark,Free,From_Centre_Bounce,Kick_Inside50,To_Ball_Up,From_Ball_Up,Rushed_Behind,Contest_Target,To_Out_On_Full,From_Out_On_Full,Error,From_Kick_In,end_x,end_y,pitch_start_x,pitch_start_y,pitch_end_x,pitch_end_y,left_right_start_x,left_right_start_y,left_right_end_x,left_right_end_y,start_distance_to_right_goal,end_distance_to_right_goal,Inside50,1,2,3,4,Duration,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,contested,mark,outcome_type,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Spoil_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,left_right_start_x_a0,left_right_start_y_a0,left_right_end_x_a0,left_right_end_y_a0,dx_a0,dy_a0,movement_a0,contested_a0,mark_a0,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Spoil_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,left_right_start_x_a1,left_right_start_y_a1,left_right_end_x_a1,left_right_end_y_a1,dx_a1,dy_a1,movement_a1,contested_a1,mark_a1,type_Kick_a2,type_Handball_a2,type_Carry_a2,type_Spoil_a2,outcome_effective_a2,outcome_ineffective_a2,outcome_clanger_a2,type_Kick_outcome_effective_a2,type_Kick_outcome_ineffective_a2,type_Kick_outcome_clanger_a2,type_Handball_outcome_effective_a2,type_Handball_outcome_ineffective_a2,type_Handball_outcome_clanger_a2,type_Carry_outcome_effective_a2,type_Carry_outcome_ineffective_a2,type_Carry_outcome_clanger_a2,type_Spoil_outcome_effective_a2,type_Spoil_outcome_ineffective_a2,type_Spoil_outcome_clanger_a2,quarter_a2,quarter_seconds_a2,overall_seconds_a2,left_right_start_x_a2,left_right_start_y_a2,left_right_end_x_a2,left_right_end_y_a2,dx_a2,dy_a2,movement_a2,contested_a2,mark_a2,team_1,team_2,time_delta1,time_delta2,dx_a01,dy_a01,move_a01,dx_a02,dy_a02,move_a02,goalscore_team,goalscore_opponent,goalscore_diff,exp_scores_label,exp_concedes_label,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
627796,1,centreBounce,miss,2.0,1,0,5.0,Sydney,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Gather From Hitout,-1.0,2.0,NaN,NaN,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022.0,2022,NaN,NaN,0,0,Centre Bounce,False,True,False,False,False,Gathe

In [73]:
match_chain.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.171135,835.862816,120.976690,227.298736,224.775711,2.523025
Sydney,42.869727,423.941544,155.817174,102.628981,97.650648,4.978334


In [74]:
match_shots = match_chain[match_chain['xScore'] > 0]
match_shots.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.171135,109.189274,6.969513,69.266295,74.726644,-5.460348
Sydney,42.869727,40.542340,2.587122,18.866929,20.643688,-1.776759


In [77]:
shots = exp_vaep_data[exp_vaep_data['xScore'] > 0]
shots[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum()

xScore                 102332.926780
exp_scores             101168.618110
exp_concedes             5956.780869
exp_vaep_value          57950.858446
exp_offensive_value     62435.993429
exp_defensive_value     -4485.134979
dtype: float64

In [78]:
exp_vaep_data[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum()

xScore                 102332.926780
exp_scores             814886.778911
exp_concedes           183148.595864
exp_vaep_value         217679.085769
exp_offensive_value    213415.507172
exp_defensive_value      4263.578565
dtype: float64

In [79]:
exp_vaep_data.groupby('match_id').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

TypeError: unsupported operand type(s) for +: 'int' and 'str'